In [ ]:
import logging
import asyncio
import polars as pl
import plotly.graph_objects as go

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.analytics.visualizations.charts import Study

from tastytrade.providers.market import MarketDataProvider
from tastytrade.messaging.models.events import CandleEvent

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(None)

logging.getLogger().handlers.clear()

TEST = True

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials().url, token=influx_user.token, org=influx_user.org
)

streamer = MarketDataProvider(dxlink, influxdb)

In [3]:
event_symbol = "SPX{=5m}"

In [4]:
start = datetime(2025, 2, 21, 9) + timedelta(hours=5)
stop = datetime(2025, 2, 21, 16) + timedelta(hours=5)

candles: pl.DataFrame = streamer.download(
    symbol=event_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=event_symbol.partition("{")[0] + "{=d}",
            start=start + timedelta(days=-5),
            stop=start,
            debug_mode=True,
        )
        .filter(pl.col("time") < start.date())
        .tail(1)
        .to_dicts()[0]
    )
)

In [ ]:
import polars as pl
import numpy as np
from plotly.subplots import make_subplots


def ema_with_seed(values: np.ndarray, length: int, seed: float) -> np.ndarray:
    alpha = 2.0 / (length + 1.0)
    out = np.zeros_like(values, dtype=float)
    if len(values) == 0:
        return out

    # Seed the first EMA
    out[0] = alpha * values[0] + (1 - alpha) * seed
    # Compute forward
    for i in range(1, len(values)):
        out[i] = alpha * values[i] + (1 - alpha) * out[i - 1]

    return out


def compute_macd_single_day(
    df_day: pl.DataFrame,
    prior_close: float,
    fast_length: int = 12,
    slow_length: int = 26,
    macd_length: int = 9,
) -> pl.DataFrame:
    # Sort by time just to be safe
    df_day = df_day.sort("time")

    # Convert 'close' to numpy
    close_np = df_day["close"].to_numpy()

    # 1) Fast EMA (seeded by prior_close)
    ema_fast = ema_with_seed(close_np, fast_length, seed=prior_close)
    # 2) Slow EMA (seeded by prior_close)
    ema_slow = ema_with_seed(close_np, slow_length, seed=prior_close)
    # MACD Value line
    value = ema_fast - ema_slow

    # 3) Signal line = EMA of Value (seed with 0.0 or some small guess).
    #    If you prefer to seed it with prior day's final MACD, you can do so;
    #    but the simplest is to seed with 0.0 or the difference from prior_close.
    signal_seed = 0.0
    ema_signal = ema_with_seed(value, macd_length, seed=signal_seed)

    # 4) Histogram
    diff = value - ema_signal
    # Calculate diff colors based on value and previous value
    diff_colors = np.empty(len(diff), dtype=object)

    for i in range(len(diff)):
        if i == 0:  # First value
            if diff[i] > 0:
                diff_colors[i] = '#04FE00'  # Bright green for first positive
            else:
                diff_colors[i] = "#FE0000"  # Bright red for first negative
        else:  # All other values
            if diff[i] > 0:  # Positive values
                if diff[i] > diff[i-1]:
                    diff_colors[i] = '#04FE00'  # Bright green for increasing positive
                else:
                    diff_colors[i] = "#006401"  # Dark green for decreasing positive
            else:  # Negative values
                if diff[i] < diff[i-1]:
                    diff_colors[i] = "#FE0000"  # Bright red for decreasing negative
                else:
                    diff_colors[i] = "#7E0100"  # Dark red for increasing negative
    # Attach them as new columns
    df_res = df_day.with_columns(
        [
            pl.Series("Value", value),
            pl.Series("avg", ema_signal),
            pl.Series("diff", diff),
            pl.Series("diff_color", diff_colors),
        ]
    )
    return df_res


def plot_macd_with_hull(df: pl.DataFrame, pad_value: float | None = None) -> None:
    # First compute the Hull MA
    hma_study = hull(input_df=df, pad_value=pad_value)

    try:
        event_symbol = df["eventSymbol"].unique()[0]
    except Exception:
        event_symbol = "Stock_Symbol"

    # Create the subplots
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, row_heights=[0.4, 0.2]
    )

    # --- Top subplot: Candlestick chart ---
    fig.add_trace(
        go.Candlestick(
            x=df["time"],
            open=df["open"],
            high=df["high"],
            low=df["low"],
            close=df["close"],
            name="Price",
            showlegend=False,
            increasing_line_color="#4CAF50",  # Green outline for up candles
            decreasing_line_color="#EF5350",  # Red outline for down candles
            increasing_fillcolor="#000000",  # Solid black fill for up candles
            decreasing_fillcolor="#EF5350",  # Solid red fill for down candles
            line_width=.75,  # Width of the candlewicks
        ),
        row=1,
        col=1,
    )
    # Create separate traces for each color segment of HMA
    for i in range(1, len(hma_study)):
        fig.add_trace(
            go.Scatter(
                x=hma_study["time"].iloc[i - 1 : i + 1],
                y=hma_study["HMA"].iloc[i - 1 : i + 1],
                mode="lines",
                line=dict(
                    color="#01FFFF" if hma_study["HMA_color"].iloc[i] == "Up" else "#FF66FE",
                    width=0.6,
                ),
                showlegend=False,
                name="HMA",
            ),
            row=1,
            col=1,
        )

    # --- Bottom subplot: MACD Value, Signal, Histogram ---
    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["Value"],
            mode="lines",
            name="MACD (Value)",
            line=dict(color="#01FFFF", width=1),
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=df["time"],
            y=df["avg"],
            mode="lines",
            name="Signal (avg)",
            line=dict(color="#F8E9A6", width=1),
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    # Histogram bars
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df["diff"],
            name="Histogram (diff)",
            marker_color=[color for color in df["diff_color"]],
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    # Zero line for MACD
    fig.add_shape(
        type="line",
        x0=df["time"][0],
        x1=df["time"][-1],
        y0=0,
        y1=0,
        line=dict(color="gray", width=1, dash="dot"),
        xref="x2",
        yref="y2",
    )

    # Update layout
    fig.update_layout(
        title=f"{event_symbol} w/ HMA-20",
        xaxis2_title="Time",
        yaxis_title="Price",
        yaxis2_title="MACD",
        showlegend=True,
        height=800,
        template="plotly_dark",
        xaxis_rangeslider_visible=False,
        plot_bgcolor="rgb(25,25,25)",
        paper_bgcolor="rgb(25,25,25)",
    )

    fig.update_yaxes(gridcolor="rgba(128,128,128,0.1)", zerolinecolor="rgba(128,128,128,0.1)")
    fig.update_xaxes(gridcolor="rgba(128,128,128,0.1)", zerolinecolor="rgba(128,128,128,0.1)")

    fig.show()


# Usage:
df_macd = compute_macd_single_day(
    candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

plot_macd_with_hull(df_macd, pad_value=prior_day.close)

# Comparison

In [ ]:
from IPython.display import Image

match event_symbol:
    case "SPX{=m}":
        display(Image(filename="/workspace/devtools/images/macd_1m_sample.png", width=850))
    case "SPX{=5m}":
        display(Image(filename="/workspace/devtools/images/macd_5m_sample.png", width=850))
    case "SPX{=15m}":
        display(Image(filename="/workspace/devtools/images/macd_15m_sample.png", width=850))
    case _:
        raise ValueError(f"No sample image for {event_symbol}")

In [ ]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=candles["time"].dt.replace_time_zone("UTC").dt.convert_time_zone("America/New_York"),
            open=candles["open"],
            high=candles["high"],
            low=candles["low"],
            close=candles["close"],
        )
    ]
)
fig.update_layout(height=800, xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
hull(input_df=candles, pad_value=prior_day.close).head(2)

In [8]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,
    params={"length": 20},
    plot_params={
        "colors": {"Up": "#01FFFF", "Down": "#FF66FE"},
        "width": 1,
    },
    value_column="HMA",
    color_column="HMA_color",
)